## 3.6 Wrangling Customers and Joi

### Contents
#### Importing Libraries
#### Importing Data
#### Data Wrangling
##### Consistency Checks
#### Import ords_prods_combined
#### Join df_combo and df_cust using 'user_id'
#### Export df_large

### Importing Libraries

In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

### Importing Data

In [3]:
# Define pathway to relevant folder
path = r'/Users/sydneyjohnson/Documents/CF Data Analytics Course/07-2024 Instacart Basket Analysis'

In [8]:
# Import customers.csv
df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

### Data Wrangling

In [13]:
df_cust.shape

(206209, 10)

In [15]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [17]:
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [19]:
# Rename 'STATE' to 'state' lowercase
df_cust.rename(columns = {'STATE' : 'state'}, inplace = True)

In [16]:
# Change 'user_id' to string
df_cust['user_id'] = df_cust['user_id'].astype('str')

In [25]:
df_cust.dtypes

user_id         object
First Name      object
Surnam          object
Gender          object
state           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [27]:
# Check for any mixed-type columns
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].map(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

First Name


In [35]:
# Investigate 'First Name' column
df_cust['First Name'].value_counts(dropna = False)

First Name
NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: count, Length: 208, dtype: int64

In [37]:
# Rename 'First Name' to 'first_name' 
df_cust.rename(columns = {'First Name' : 'first_name'}, inplace = True)

In [39]:
# Check data type for 'first_name'
df_cust['first_name'].dtype

dtype('O')

#### I am not going to impute or delete the missing data from 'first_name'. The surname should be enough information.
#### Consistency Checks

In [46]:
# Check for duplicates
df_dups = df_cust[df_cust.duplicated()]
df_dups

,user_id,first_name,Surnam,Gender,state,Age,date_joined,n_dependants,fam_status,income


In [48]:
### Exporting df_cust to "prepared_data"
df_cust.to_csv(os.path.join(path, '02 Data','Prepared Data', 'customers_clean.csv'))

## Import ords_prods_combined

In [6]:
# Import combo_7_25_correct.pkl
df_combo = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'combo_7_25_correct.pkl'))

In [10]:
df_cust = pd.read_csv(os.path.join(path, '02 Data','Prepared Data', 'customers_clean.csv'))

In [18]:
df_cust.dtypes

Unnamed: 0       int64
user_id         object
first_name      object
Surnam          object
Gender          object
state           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [26]:
df_combo.dtypes

order_id                    object
user_id                     object
order_number                 int64
orders_day_of_week           int64
order_hour_placed            int64
days_since_prior_order     float64
product_id                  object
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
max_order                    int64
loyalty_flag                object
avg_price                  float64
spending_flag               object
med_days_prior             float64
frequency_flag              object
dtype: object

In [24]:
# Change ids in df_combo to strings
df_combo[['order_id','user_id','product_id']] = df_combo[['order_id','user_id','product_id']].astype('str')

In [31]:
# Drop '_merge' from df_combo after checking it
df_combo['_merge'].value_counts(dropna = False)

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [33]:
df_combo = df_combo.drop(columns = ['_merge'])

## Join df_combo and df_cust using 'user_id'

In [35]:
df_large = df_combo.merge(df_cust, on = 'user_id', indicator = True)

In [37]:
df_large.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_placed,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,first_name,Surnam,Gender,state,Age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,2539329,1,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [39]:
df_large['_merge'].value_counts(dropna = False)

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

## Export df_large

In [41]:
## Export df_large to pickle
df_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'combo_7_30.pkl'))